## 웹 크롤링

### 인터넷 접속 라이브러리 추가

- urllib.request 모듈로 작업
- html 소스 로드로 별다른 문제없음

In [2]:
from urllib.request import urlopen, Request

# 도시별 날씨 검색함수
def get_weather(city):
    # 기상청 홈페이지 도시별 날씨 페이지
    url = 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do'
    page = urlopen(url=url)

    text = page.read().decode('utf-8')
    text = text[text.find(f'>{city}</a>'):]
    
    # 기온 가져오기 반복문 13번
    for i in range(7):
        text = text[text.find('<td>')+1:]
    

    start = 3
    end = text.find('</td>')
    current_temp = text[start:end]
    print(f'{city}의 현재 기온은 {current_temp}˚C 입니다.')

    # 습도 가져오기
    for i in range(7):
        text = text[text.find('<td>')+1:]
    

    start = 3
    end = text.find('</td>')
    current_humid = text[start:end]
    print(f'{city}의 현재 기온은 {current_humid}˚C 입니다.')


get_weather('부산')

부산의 현재 기온은 11.6˚C 입니다.
부산의 현재 기온은 <a href="city-obs.do?tm=2023.2.8.14:00&amp;type=t99&amp;mode=0&amp;reg=100&amp;auto_man=m&amp;stn=243">부안</a>˚C 입니다.


### OpenAPI 크롤링

- 공공데이터 포털
    - http://www.data.go.kr
    - 부산 광역시 버스정보안내기 현황

```python
 response = requests.get(total_url, verify = False)
 ```

- 문제점
    - https를 일반 request로 부르면 SSL 오류발생
    - 해결하려면 외부모듈 requests를 사용, verify=False 옵션을 지정
    - 아래 셀의 19번 라인!

In [12]:
import requests
from urllib.parse import quote, unquote, urlencode   # 한글을 urlencode로 변환 해줌  롯데 ->'%EB%A1%AF%EB%8D%B0%EB%B0%B1%ED%99%94%EC%A0%90' 
import json

# def getRequestUrl(url):
#     req= Request(url)

#     try :
#         res = urlopen(req)
#         if res.getcode() == 200:
#             return res.read().decode('utf-8')
#     except  Exception as e:
#         print(e)
#         return None
#stationName 정류소 이름 
def getDataPortalSearch(stationName, type):
    api_url ='https://apis.data.go.kr/6260000/BusanTblBusinfoeqStusService/getTblBusinfoeqStusInfo'
    queryString ="?" +urlencode( 
        {
            'serviceKey': 'eah4sO3GHeKxsDRxvzwAQMU5I5zDKlGRz6WPeocoIXYce8ptdka7EIB84Lod4N+hPh1UtODmsrTnXKa0Dvj/8g==',
            'pageNO': '1',
            'numOfRows':'10',
            'resultType':type,
            'stationLoc': stationName
        }
    )
    total_url= api_url +queryString
    response = requests.get(total_url, verify = False)
    return response.text
    # api_url+='?serviceKey=4n4Miwzm5p37SLb9Jk9bJa%2FMhFYSTl8mkQIensYxsOuwWyjpePzkk6oyRp3pOsd8GVnzwwQelKHMwSc0bPVfSA%3D%3D'
    # api_url+='&pageNo=1'
    # api_url+='&numOfRows=10'
    # api_url+=f'&resultType={type}'
    # api_url+=f'&stationLoc={quote(stationName)}'

    # response = getRequestUrl(api_url)

    # if response == None :
    #     return None
    # else:
    #     return json.loads(response)
try:

    result = getDataPortalSearch('백화점', 'json')
    json_data = json.loads(result)
    station_data = json_data['getTblBusinfoeqStusInfo']['body']['items']['item']

# 전체 정류소 위치값 마커 표시
    for item in station_data:
        print(item)
    
except Exception as e:
    print('찾는 데이터가 없습니다')
# quote('롯데백화점')

{'stationNum': '09283', 'stationLoc': '롯데백화점센텀시티점', 'lat': '35.16967324', 'lng': '129.1317843', 'addr': '해운대구 우동 1496', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '09336', 'stationLoc': '롯데백화점센텀시티점', 'lat': '35.16997953', 'lng': '129.1320357', 'addr': '해운대구 우동 1498', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '05718', 'stationLoc': '서면역.롯데호텔백화점', 'lat': '35.15776525', 'lng': '129.0566948', 'addr': '부산진구 부전동 576-1', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '05719', 'stationLoc': '서면역.롯데호텔백화점', 'lat': '35.15748116', 'lng': '129.0561774', 'addr': '부산진구 부전동 576-1', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '06030', 'stationLoc': '롯데백화점동래점.명륜역', 'lat': '35.21276314', 'lng': '129.0781263', 'addr': '동래구 온천1동 474-25', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '06046', 'stationLoc': '롯데백화점동래점', 'lat': '35.2101971', 'lng': '129.0778307', 'addr': '동래구 온천2동 503-46', 'insYear': '', 'dataDay': '2023-02-03'}


c:\Source\studyPython2023\venv\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


### OpenAPI로 가져온 데이터 지도 표시

```python
 stop_str = '<h4>' + item['stationLoc'] + '</h4>'
    stop_str += item['stationNum'] + '<br>'
    stop_str += item['addr']
```
- 문제점
    - 딕셔너리 구조하고 문자열 포맷팅 f'{}' 호환안됨!
    - 구식방법인 문자열 결합방식으로 해결

In [21]:
import folium

if len(station_data) > 0: # 정류소 중 제일 첫번째 인덱스 정류소 위경도를 중심으로 
    center_lat = station_data[0]['lat']
    center_lnd = station_data[0]['lng']

    print(center_lat, center_lnd)
    print('데이터 존재')


m = folium.Map(location=[center_lat, center_lnd], zoom_start=13)

for item in station_data:
    stop_str = '<h4>' + item['stationLoc'] + '</h4>'
    stop_str += item['stationNum'] + '<br>'
    stop_str += item['addr']
    iframe = folium.IFrame(stop_str)
    popup = folium.Popup(iframe, min_width=200, max_width=200) # 팝업사이즈 결정
    
    folium.Marker(location=[item['lat'], item['lng']], popup=popup,
                  icon=folium.Icon(icon='pushpin')).add_to(m)

m

35.16967324 129.1317843
데이터 존재


### BeautifulSoup (version 4)

웹크롤링을 편하게 해주는 도구(라이브러리)

```
pip install beautifulsoup4
```


In [15]:
!pip install beautifulsoup4

     -------------------------------------- 129.4/129.4 kB 7.9 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### 모듈 import

```python
import BeautifulSoup
from bs4 import BeautifulSoup
```

In [24]:
from bs4 import BeautifulSoup
import requests

url = 'https://kin.naver.com/search/list.naver?query=%EC%A0%9C%EC%9E%84%EC%8A%A4+%EC%9B%B9+%EB%A7%9D%EC%9B%90%EA%B2%BD'

response = requests.get(url)
if response.status_code == 200: # webpage OK
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    #print(soup)
    # title = soup.select_one('#s_content > div.section > ul > li:nth-child(1) > dl > dt > a')
    ul = soup.select_one('ul.basic1')
    titles = ul.select('li > dl > dt > a')
    for title in titles:
        print(title.get_text())
else:
    print(f'Error : {response.status_code}')

제임스웹 우주망원경 질문이요
최고고도 부탁드립니다. 제임스웹 망원경이
제임스 웹 망원경 우주사진 원본 어디서... 
제임스 웹 망원경 촬영 원리
제임스 웹 망원경
제임스 웹 우주망원경이 찍은 첫 사진은... 
제임스 웹 망원경의 종류 알려주세요
제임스 웹 우주망원경
제임스웹 망원경
제임스 웹 우주 망원경 제작 기간이
